## Интерактивни Phi 3 Mini 4K Инструкт Четбот са Whisper

### Увод:
Интерактивни Phi 3 Mini 4K Инструкт Четбот је алат који омогућава корисницима да комуницирају са Microsoft Phi 3 Mini 4K Инструкт демо уз помоћ текстуалног или аудио уноса. Четбот се може користити за различите задатке, као што су превођење, ажурирање временских услова и прикупљање општих информација.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


[Направите свој Huggingface приступни токен](https://huggingface.co/settings/tokens)

Направите нови токен  
Унесите ново име  
Изаберите дозволе за писање  
Копирајте токен и сачувајте га на безбедном месту


Следећи Python код обавља два главна задатка: увоз модула `os` и постављање променљиве окружења.

1. Увоз модула `os`:
   - Модул `os` у Python-у пружа начин за интеракцију са оперативним системом. Омогућава вам да извршите различите задатке повезане са оперативним системом, као што су приступ променљивим окружења, рад са датотекама и директоријумима, итд.
   - У овом коду, модул `os` се увози помоћу наредбе `import`. Ова наредба омогућава функционалност модула `os` за употребу у тренутном Python скрипту.

2. Постављање променљиве окружења:
   - Променљива окружења је вредност којој програми који раде на оперативном систему могу приступити. То је начин за чување конфигурационих подешавања или других информација које могу користити више програма.
   - У овом коду, нова променљива окружења се поставља помоћу речника `os.environ`. Кључ речника је `'HF_TOKEN'`, а вредност се додељује из променљиве `HUGGINGFACE_TOKEN`.
   - Променљива `HUGGINGFACE_TOKEN` је дефинисана непосредно изнад овог исечка кода и додељује јој се стринг вредност `"hf_**************"` помоћу синтаксе `#@param`. Ова синтакса се често користи у Jupyter нотебуковима како би се омогућио унос корисника и конфигурација параметара директно у интерфејсу нотебука.
   - Постављањем променљиве окружења `'HF_TOKEN'`, она може бити доступна другим деловима програма или другим програмима који раде на истом оперативном систему.

Укратко, овај код увози модул `os` и поставља променљиву окружења под именом `'HF_TOKEN'` са вредношћу која је наведена у променљивој `HUGGINGFACE_TOKEN`.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

Овај кодни исечак дефинише функцију под називом clear_output која се користи за брисање излазних података тренутне ћелије у Jupyter Notebook-у или IPython-у. Хајде да разложимо код и разумемо његову функционалност:

Функција clear_output прихвата један параметар под називом wait, који је булова вредност. Подразумевано, wait је постављен на False. Овај параметар одређује да ли функција треба да сачека док нови излаз не буде доступан за замену постојећег излаза пре него што га обрише.

Сама функција се користи за брисање излазних података тренутне ћелије. У Jupyter Notebook-у или IPython-у, када ћелија генерише излаз, као што је одштампани текст или графички прикази, тај излаз се приказује испод ћелије. Функција clear_output омогућава брисање тог излаза.

Имплементација функције није дата у кодном исечку, што је назначено елипсом (...). Елипса представља место за стварни код који извршава брисање излазних података. Имплементација функције може укључивати интеракцију са Jupyter Notebook или IPython API-јем ради уклањања постојећег излаза из ћелије.

Укратко, ова функција пружа практичан начин за брисање излазних података тренутне ћелије у Jupyter Notebook-у или IPython-у, чиме се олакшава управљање и ажурирање приказаног излаза током интерактивних сесија кодирања.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Извршите текст-у-говор (TTS) користећи Edge TTS услугу. Хајде да прођемо кроз релевантне имплементације функција једну по једну:

1. `calculate_rate_string(input_value)`: Ова функција узима улазну вредност и израчунава стринг брзине за TTS глас. Улазна вредност представља жељену брзину говора, где вредност 1 представља нормалну брзину. Функција израчунава стринг брзине одузимањем 1 од улазне вредности, множењем са 100, и затим одређује знак на основу тога да ли је улазна вредност већа или једнака 1. Функција враћа стринг брзине у формату "{sign}{rate}".

2. `make_chunks(input_text, language)`: Ова функција узима улазни текст и језик као параметре. Делује тако што дели улазни текст на делове на основу језичких правила. У овој имплементацији, ако је језик "English", функција дели текст на свакој тачки ("."), уклањајући било који водећи или завршни размак. Затим додаје тачку на крај сваког дела и враћа филтрирану листу делова.

3. `tts_file_name(text)`: Ова функција генерише име датотеке за TTS аудио на основу улазног текста. Извршава неколико трансформација на тексту: уклања завршну тачку (ако постоји), претвара текст у мала слова, уклања водеће и завршне размаке, и замењује размаке са доњим цртама. Затим скраћује текст на максимално 25 карактера (ако је дужи) или користи цео текст ако је празан. На крају, генерише насумични стринг користећи модул [`uuid`] и комбинује га са скраћеним текстом да би креирала име датотеке у формату "/content/edge_tts_voice/{truncated_text}_{random_string}.mp3".

4. `merge_audio_files(audio_paths, output_path)`: Ова функција спаја више аудио датотека у једну аудио датотеку. Узима листу путања аудио датотека и излазну путању као параметре. Функција иницијализује празан објекат `AudioSegment` назван [`merged_audio`]. Затим пролази кроз сваку путању аудио датотеке, учитава аудио датотеку користећи метод `AudioSegment.from_file()` из библиотеке `pydub`, и додаје тренутну аудио датотеку у објекат [`merged_audio`]. На крају, извози спојени аудио у специфицирану излазну путању у MP3 формату.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`: Ова функција извршава TTS операцију користећи Edge TTS услугу. Узима листу текстуалних делова, брзину говора, име гласа и путању за чување као параметре. Ако је број делова већи од 1, функција креира директоријум за чување појединачних аудио датотека делова. Затим пролази кроз сваки део, конструише Edge TTS команду користећи функцију `calculate_rate_string()`, име гласа и текст дела, и извршава команду користећи функцију `os.system()`. Ако је извршење команде успешно, додаје путању генерисане аудио датотеке у листу. Након обраде свих делова, спаја појединачне аудио датотеке користећи функцију `merge_audio_files()` и чува спојени аудио у специфицираној путањи за чување. Ако постоји само један део, директно генерише Edge TTS команду и чува аудио у путањи за чување. На крају, враћа путању за чување генерисане аудио датотеке.

6. `random_audio_name_generate()`: Ова функција генерише насумично име аудио датотеке користећи модул [`uuid`]. Генерише насумични UUID, претвара га у стринг, узима првих 8 карактера, додаје екстензију ".mp3", и враћа насумично име аудио датотеке.

7. `talk(input_text)`: Ова функција је главна тачка уласка за извршавање TTS операције. Узима улазни текст као параметар. Прво проверава дужину улазног текста да би утврдила да ли је реч о дугачкој реченици (већој или једнакој 600 карактера). На основу дужине и вредности променљиве `translate_text_flag`, одређује језик и генерише листу текстуалних делова користећи функцију `make_chunks()`. Затим генерише путању за чување аудио датотеке користећи функцију `random_audio_name_generate()`. На крају, позива функцију `edge_free_tts()` да изврши TTS операцију и враћа путању за чување генерисане аудио датотеке.

Укратко, ове функције раде заједно да би поделиле улазни текст на делове, генерисале име датотеке за аудио, извршиле TTS операцију користећи Edge TTS услугу, и спојиле појединачне аудио датотеке у једну аудио датотеку.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

Имплементација две функције: convert_to_text и run_text_prompt, као и декларација две класе: str и Audio.

Функција convert_to_text узима audio_path као улаз и транскрибује аудио у текст користећи модел назван whisper_model. Функција прво проверава да ли је gpu флаг постављен на True. Ако јесте, whisper_model се користи са одређеним параметрима као што су word_timestamps=True, fp16=True, language='English', и task='translate'. Ако је gpu флаг False, whisper_model се користи са fp16=False. Добијена транскрипција се затим чува у датотеци названој 'scan.txt' и враћа као текст.

Функција run_text_prompt узима message и chat_history као улаз. Користи функцију phi_demo за генерисање одговора од чатбота на основу улазне поруке. Генерисани одговор се затим прослеђује функцији talk, која конвертује одговор у аудио датотеку и враћа путању до те датотеке. Класа Audio се користи за приказивање и репродукцију аудио датотеке. Аудио се приказује користећи функцију display из модула IPython.display, а објекат Audio се креира са параметром autoplay=True, тако да аудио аутоматски почиње да се репродукује. chat_history се ажурира са улазном поруком и генерисаним одговором, а враћају се празан стринг и ажурирани chat_history.

Класа str је уграђена класа у Python-у која представља секвенцу карактера. Пружа различите методе за манипулацију и рад са стринговима, као што су capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill, и још много тога. Ове методе омогућавају извршавање операција као што су претрага, замена, форматирање и манипулација стринговима.

Класа Audio је прилагођена класа која представља аудио објекат. Користи се за креирање аудио плејера у Jupyter Notebook окружењу. Класа прихвата различите параметре као што су data, filename, url, embed, rate, autoplay, и normalize. Параметар data може бити numpy низ, листа узорака, стринг који представља име датотеке или URL, или сирови PCM подаци. Параметар filename се користи за спецификацију локалне датотеке из које се учитавају аудио подаци, а параметар url се користи за спецификацију URL-а са којег се преузимају аудио подаци. Параметар embed одређује да ли аудио подаци треба да буду уграђени користећи data URI или референцирани из оригиналног извора. Параметар rate спецификује брзину узорковања аудио података. Параметар autoplay одређује да ли аудио треба аутоматски да почне да се репродукује. Параметар normalize спецификује да ли аудио подаци треба да буду нормализовани (преразмерени) на максимални могући опсег. Класа Audio такође пружа методе као што је reload за поновно учитавање аудио података из датотеке или URL-а, и атрибуте као што су src_attr, autoplay_attr, и element_id_attr за преузимање одговарајућих атрибута за аудио елемент у HTML-у.

Укратко, ове функције и класе се користе за транскрипцију аудио записа у текст, генерисање аудио одговора од чатбота, и приказивање и репродукцију аудио записа у Jupyter Notebook окружењу.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако се трудимо да превод буде тачан, молимо вас да имате у виду да аутоматизовани преводи могу садржати грешке или нетачности. Оригинални документ на његовом изворном језику треба сматрати меродавним извором. За критичне информације препоручује се професионални превод од стране људи. Не преузимамо одговорност за било каква погрешна тумачења или неспоразуме који могу настати услед коришћења овог превода.
